In [101]:
import os
import json
import pandas as pd
import collections
from datetime import datetime

In [102]:
dir_path = "/content/drive/MyDrive/dataset/Events"

# Preprocesse

In [ ]:
import os
import shutil

# Iterate over all items in the directory
for item in os.listdir(dir_path):
    item_path = os.path.join(dir_path, item)

    # Check if the item is a directory and if its name starts with "Subject_"
    if os.path.isdir(item_path) and item.startswith("Subject_"):
        try:
            # Extract the subject number
            subject_number = item[len("Subject_"):]

            # Create the new directory name
            new_dir_name = os.path.join(dir_path, subject_number)

            # Rename the directory
            os.rename(item_path, new_dir_name)
            print(f"Renamed directory: {item_path} to {new_dir_name}")

        except OSError as e:
            print(f"Error renaming directory {item_path}: {e}")
    else:
        print(f"Directory {item} skipped")

# Extract to middle JSON file

In [103]:
END_EVENT = [3, 4, 5, 7, 8, 9, 11, 12]

In [110]:
def extract_start_end(df_event):
  end_with_lst = []
  start_with_lst = [0]

  for index, row in df_event.iterrows():
    # print("Row:", index, " Event:", row['Event Label'])
    if int(row["Event Label"]) in END_EVENT:
      END = True
      if round(row["Event Label"] - int(row["Event Label"]), 1) == 0.2:
        if len(end_with_lst) > 0 and round(row["Event Label"] - df_event.iloc[end_with_lst[-1]]["Event Label"], 1) == 0.1:
          end_with_lst.pop()
      end_with_lst.append(index)

  for idx in end_with_lst:
    if idx + 1 < len(df_event):
      start_with_lst.append(idx + 1)

  print("extract_start_end to list")
  return start_with_lst, end_with_lst

In [111]:
def extract_to_dict(start_with_lst, end_with_lst):
  sequence = collections.defaultdict(list)
  for i in range(len(start_with_lst)):
    idx_sart = start_with_lst[i]
    idx_end = end_with_lst[i]

    lst = df_event.iloc[idx_sart : idx_end + 1].values.tolist()
    sequence[i + 1] = lst

  print(f"Sequences extracted!")
  return sequence

In [112]:
def export_to_json(sequence, file_path, st, ss):
  # Convert the sequence dictionary to JSON format
  json_data = json.dumps(sequence, indent=4)

  # Specify the file path where you want to save the JSON data
  file_path = dir_path + f"/Subject_{st}_{ss}_sequences.json"  # Replace with your desired path

  # Write the JSON data to the file
  with open(file_path, 'w') as json_file:
      json_file.write(json_data)

  print(f"Sequences saved to {file_path}")

In [113]:
# R (rotation): 10.1, 10.2
# action: end_event_#_(R)
# hold_time = drop_first_time - pick_last_time
# duration = drop_first_time - sequence_start_time
# pick_last_time: 2.2 (2.1), 6.2 (6.1)
# drop_first_time: 4.1, 5.1, 8.1, 9.1

def extract_to_df(sequence):
  columns = ["action", "sequence_start_time", "pick_last_time", "drop_first_time", "sequence_end_time", "hold_time", "duration"]
  df_seq = pd.DataFrame(columns=columns)
  for n, seq_lst in sequence.items():
    print(n, seq_lst)
    action, sequence_start_time, pick_last_time, drop_first_time, sequence_end_time, hold_time, duration = None, datetime.strptime(seq_lst[0][1], '%Y-%m-%d %H:%M:%S'), None, None, datetime.strptime(seq_lst[-1][1], '%Y-%m-%d %H:%M:%S'), None, None
    ROTATION = False
    for event, dt in seq_lst:
      print(event, dt)
      dt_object = datetime.strptime(dt, '%Y-%m-%d %H:%M:%S')
      if event in [2.1, 2.2, 6.1, 6.2]:
        pick_last_time = dt_object
      if event in [4.1, 5.1, 8.1, 9.1]:
        drop_first_time = dt_object

      if event in [10.1, 10.2]:
        ROTATION = True

    if ROTATION:
      action = f"end_event_{event}_R"
    else:
      action = f"end_event_{event}"

    print(action, sequence_start_time, pick_last_time, drop_first_time, sequence_end_time)
    if drop_first_time and pick_last_time:
      hold_time = int((drop_first_time - pick_last_time).total_seconds())
    if drop_first_time and sequence_start_time:
      duration = int((drop_first_time - sequence_start_time).total_seconds())

    row = [action, sequence_start_time, pick_last_time, drop_first_time, sequence_end_time, hold_time, duration]
    df_seq.loc[len(df_seq)] = row

  print("Final Table Finished!")
  return df_seq

## Exception


*   6_1
*   67_2



In [120]:
for st in range(1, 74):
  for ss in range(1, 4):
    event_file = dir_path + f"/{st}/Subject_{st}_{ss}_event_log.xlsx"
    if os.path.exists(event_file):
      df_event = pd.read_excel(event_file, usecols=[0, 1])
      df_event.columns = ["Event Label", "Timestamp (Real Time)"]

      # Convert Timestamp column to datetime objects
      df_event["Timestamp (Real Time)"] = pd.to_datetime(df_event["Timestamp (Real Time)"], format="%m/%d/%Y %H:%M:%S", errors='coerce')
      # Format the Timestamp column to the desired format
      df_event["Timestamp (Real Time)"] = df_event["Timestamp (Real Time)"].dt.strftime("%Y-%m-%d %H:%M:%S")

      print("st", st, " ss ", ss)
      # Remove rows where both 'Event Label' and 'Event Timestamp (Real Time)' are NaN
      df_event = df_event.dropna(subset=['Event Label', 'Timestamp (Real Time)'], how='all')

      start_with_lst, end_with_lst = extract_start_end(df_event)
      # print(end_with_lst, len(end_with_lst))
      # print(start_with_lst, len(start_with_lst))
      len_min = min(len(start_with_lst), len(end_with_lst))
      start_with_lst = start_with_lst[:len_min]
      end_with_lst = end_with_lst[:len_min]

      extract_to_dict(start_with_lst, end_with_lst)
      sequence = extract_to_dict(start_with_lst, end_with_lst)
      export_to_json(sequence, event_file, st, ss)
      df_seq = extract_to_df(sequence)
      # print(df_seq.tail(2))
      df_seq.to_excel(dir_path + f"/Subject_{st}_{ss}_actions_datetime.xlsx")

    else:
      # print(f"File {event_file} does not exist")
      pass


st 1  ss  1
extract_start_end to list
Sequences extracted!
Sequences extracted!
Sequences saved to /content/drive/MyDrive/dataset/Events/Subject_1_1_sequences.json
1 [[0.0, '2022-09-01 14:51:55'], [1.0, '2022-09-01 14:51:56'], [2.1, '2022-09-01 14:51:57'], [4.1, '2022-09-01 14:52:07']]
0.0 2022-09-01 14:51:55
1.0 2022-09-01 14:51:56
2.1 2022-09-01 14:51:57
4.1 2022-09-01 14:52:07
end_event_4.1 2022-09-01 14:51:55 2022-09-01 14:51:57 2022-09-01 14:52:07 2022-09-01 14:52:07
2 [[0.0, '2022-09-01 14:52:13'], [1.0, '2022-09-01 14:52:13'], [2.1, '2022-09-01 14:52:13'], [4.1, '2022-09-01 14:52:20']]
0.0 2022-09-01 14:52:13
1.0 2022-09-01 14:52:13
2.1 2022-09-01 14:52:13
4.1 2022-09-01 14:52:20
end_event_4.1 2022-09-01 14:52:13 2022-09-01 14:52:13 2022-09-01 14:52:20 2022-09-01 14:52:20
3 [[0.0, '2022-09-01 14:54:05'], [1.0, '2022-09-01 14:54:08'], [2.1, '2022-09-01 14:54:08'], [4.1, '2022-09-01 14:54:20']]
0.0 2022-09-01 14:54:05
1.0 2022-09-01 14:54:08
2.1 2022-09-01 14:54:08
4.1 2022-09-01 

<ipython-input-113-77cd221cec25>:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_seq.loc[len(df_seq)] = row
<ipython-input-113-77cd221cec25>:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_seq.loc[len(df_seq)] = row
<ipython-input-113-77cd221cec25>:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, e

st 1  ss  3
extract_start_end to list
Sequences extracted!
Sequences extracted!
Sequences saved to /content/drive/MyDrive/dataset/Events/Subject_1_3_sequences.json
1 [[0.0, '2022-09-01 15:20:34'], [1.0, '2022-09-01 15:20:34'], [2.1, '2022-09-01 15:20:35'], [4.1, '2022-09-01 15:20:50']]
0.0 2022-09-01 15:20:34
1.0 2022-09-01 15:20:34
2.1 2022-09-01 15:20:35
4.1 2022-09-01 15:20:50
end_event_4.1 2022-09-01 15:20:34 2022-09-01 15:20:35 2022-09-01 15:20:50 2022-09-01 15:20:50
2 [[0.0, '2022-09-01 15:20:53'], [1.0, '2022-09-01 15:21:05'], [10.1, '2022-09-01 15:21:19'], [10.2, '2022-09-01 15:21:23'], [3.0, '2022-09-01 15:21:24']]
0.0 2022-09-01 15:20:53
1.0 2022-09-01 15:21:05
10.1 2022-09-01 15:21:19
10.2 2022-09-01 15:21:23
3.0 2022-09-01 15:21:24
end_event_3.0_R 2022-09-01 15:20:53 None None 2022-09-01 15:21:24
3 [[0.0, '2022-09-01 15:21:25'], [1.0, '2022-09-01 15:21:26'], [2.1, '2022-09-01 15:21:26'], [3.0, '2022-09-01 15:21:33']]
0.0 2022-09-01 15:21:25
1.0 2022-09-01 15:21:26
2.1 2022-

<ipython-input-113-77cd221cec25>:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_seq.loc[len(df_seq)] = row
<ipython-input-113-77cd221cec25>:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_seq.loc[len(df_seq)] = row
<ipython-input-113-77cd221cec25>:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, e

st 2  ss  2
extract_start_end to list
Sequences extracted!
Sequences extracted!
Sequences saved to /content/drive/MyDrive/dataset/Events/Subject_2_2_sequences.json
1 [[0.0, '2022-09-05 14:19:57'], [1.0, '2022-09-05 14:19:57'], [2.1, '2022-09-05 14:19:57'], [4.1, '2022-09-05 14:20:25']]
0.0 2022-09-05 14:19:57
1.0 2022-09-05 14:19:57
2.1 2022-09-05 14:19:57
4.1 2022-09-05 14:20:25
end_event_4.1 2022-09-05 14:19:57 2022-09-05 14:19:57 2022-09-05 14:20:25 2022-09-05 14:20:25
2 [[0.0, '2022-09-05 14:20:31'], [1.0, '2022-09-05 14:20:31'], [2.1, '2022-09-05 14:20:32'], [10.1, '2022-09-05 14:20:45'], [10.2, '2022-09-05 14:20:57'], [5.1, '2022-09-05 14:20:58']]
0.0 2022-09-05 14:20:31
1.0 2022-09-05 14:20:31
2.1 2022-09-05 14:20:32
10.1 2022-09-05 14:20:45
10.2 2022-09-05 14:20:57
5.1 2022-09-05 14:20:58
end_event_5.1_R 2022-09-05 14:20:31 2022-09-05 14:20:32 2022-09-05 14:20:58 2022-09-05 14:20:58
Final Table Finished!
st 2  ss  3
extract_start_end to list
Sequences extracted!
Sequences extra

...

<ipython-input-113-77cd221cec25>:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_seq.loc[len(df_seq)] = row
<ipython-input-113-77cd221cec25>:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_seq.loc[len(df_seq)] = row
<ipython-input-113-77cd221cec25>:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, e

Sequences saved to /content/drive/MyDrive/dataset/Events/Subject_72_2_sequences.json
1 [[0.0, '2022-12-07 13:49:00'], [2.1, '2022-12-07 13:49:01'], [10.1, '2022-12-07 13:49:20'], [10.2, '2022-12-07 13:49:27'], [5.1, '2022-12-07 13:49:29']]
0.0 2022-12-07 13:49:00
2.1 2022-12-07 13:49:01
10.1 2022-12-07 13:49:20
10.2 2022-12-07 13:49:27
5.1 2022-12-07 13:49:29
end_event_5.1_R 2022-12-07 13:49:00 2022-12-07 13:49:01 2022-12-07 13:49:29 2022-12-07 13:49:29
2 [[2.1, '2022-12-07 13:49:38'], [3.0, '2022-12-07 13:49:43']]
2.1 2022-12-07 13:49:38
3.0 2022-12-07 13:49:43
end_event_3.0 2022-12-07 13:49:38 2022-12-07 13:49:38 None 2022-12-07 13:49:43
3 [[6.1, '2022-12-07 13:49:44'], [7.0, '2022-12-07 13:49:47']]
6.1 2022-12-07 13:49:44
7.0 2022-12-07 13:49:47
end_event_7.0 2022-12-07 13:49:44 2022-12-07 13:49:44 None 2022-12-07 13:49:47
4 [[2.1, '2022-12-07 13:49:50'], [10.1, '2022-12-07 13:49:55'], [10.2, '2022-12-07 13:49:58'], [3.0, '2022-12-07 13:49:58']]
2.1 2022-12-07 13:49:50
10.1 2022-12-

<ipython-input-113-77cd221cec25>:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_seq.loc[len(df_seq)] = row
<ipython-input-113-77cd221cec25>:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_seq.loc[len(df_seq)] = row
<ipython-input-113-77cd221cec25>:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, e

st 73  ss  2
extract_start_end to list
Sequences extracted!
Sequences extracted!
Sequences saved to /content/drive/MyDrive/dataset/Events/Subject_73_2_sequences.json
1 [[0.0, '2023-01-23 14:39:35'], [1.0, '2023-01-23 14:39:35'], [2.1, '2023-01-23 14:39:36'], [10.1, '2023-01-23 14:39:39'], [10.2, '2023-01-23 14:39:41'], [4.1, '2023-01-23 14:39:43']]
0.0 2023-01-23 14:39:35
1.0 2023-01-23 14:39:35
2.1 2023-01-23 14:39:36
10.1 2023-01-23 14:39:39
10.2 2023-01-23 14:39:41
4.1 2023-01-23 14:39:43
end_event_4.1_R 2023-01-23 14:39:35 2023-01-23 14:39:36 2023-01-23 14:39:43 2023-01-23 14:39:43
2 [[1.0, '2023-01-23 14:39:48'], [2.1, '2023-01-23 14:39:49'], [10.1, '2023-01-23 14:39:49'], [10.2, '2023-01-23 14:39:52'], [4.1, '2023-01-23 14:39:52'], [4.2, '2023-01-23 14:39:54']]
1.0 2023-01-23 14:39:48
2.1 2023-01-23 14:39:49
10.1 2023-01-23 14:39:49
10.2 2023-01-23 14:39:52
4.1 2023-01-23 14:39:52
4.2 2023-01-23 14:39:54
end_event_4.2_R 2023-01-23 14:39:48 2023-01-23 14:39:49 2023-01-23 14:39:52 

<ipython-input-113-77cd221cec25>:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_seq.loc[len(df_seq)] = row
<ipython-input-113-77cd221cec25>:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_seq.loc[len(df_seq)] = row
<ipython-input-113-77cd221cec25>:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, e